In [16]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from pprint import pprint


In [2]:
df = pd.read_csv('EV_sales.csv')

### EV Car Sales Between 2010 and 2024

#### About this file:
- region: Geographical area where the data was collected (e.g., Australia)
- category: Nature of the data (e.g., Historical, Projection)
- parameter: Type of metric (e.g., EV sales, EV stock share)
- mode: Mode of transport (e.g., Cars)
- powertrain: Type of EV: BEV (Battery Electric Vehicle), PHEV (Plug-in Hybrid - EV), EV
- year: Year of the data point (e.g., 2011, 2012)
- unit: Measurement unit (e.g., Vehicles, percent)
- value: Actual recorded value

In [3]:
df.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
1,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
2,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000


The first step, after getting a general idea of how the DF looks, is to drop the data that's not directly relevant or that doesn't provide any meaningful insight, and to leave only those that are worth studying deeper.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      3798 non-null   object 
 1   category    3798 non-null   object 
 2   parameter   3798 non-null   object 
 3   mode        3798 non-null   object 
 4   powertrain  3798 non-null   object 
 5   year        3798 non-null   int64  
 6   unit        3798 non-null   object 
 7   value       3798 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 237.5+ KB


In [23]:
columns = ['region', 'category', 'parameter', 'mode', 'powertrain', 'year', 'unit']

for column in columns:
    print(f"{column}'s unique values:\n{df[column].unique()}\n")

region's unique values:
['Australia' 'Austria' 'Belgium' 'Brazil' 'Bulgaria' 'Canada' 'Chile'
 'China' 'Colombia' 'Costa Rica' 'Croatia' 'Cyprus' 'Czech Republic'
 'Denmark' 'Estonia' 'EU27' 'Europe' 'Finland' 'France' 'Germany' 'Greece'
 'Hungary' 'Iceland' 'India' 'Ireland' 'Israel' 'Italy' 'Japan' 'Korea'
 'Latvia' 'Lithuania' 'Luxembourg' 'Mexico' 'Netherlands' 'New Zealand'
 'Norway' 'Poland' 'Portugal' 'Rest of the world' 'Romania' 'Seychelles'
 'Slovakia' 'Slovenia' 'South Africa' 'Spain' 'Sweden' 'Switzerland'
 'Turkiye' 'United Arab Emirates' 'United Kingdom' 'USA' 'World']

category's unique values:
['Historical']

parameter's unique values:
['EV sales' 'EV stock share' 'EV sales share' 'EV stock'
 'Electricity demand' 'Oil displacement Mbd'
 'Oil displacement, million lge']

mode's unique values:
['Cars']

powertrain's unique values:
['BEV' 'EV' 'PHEV' 'FCEV']

year's unique values:
[2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2010]

unit's unique values

Since both the "category" and the "mode" columns do not add any meaningful information to the database, it is safe to say we can drop them entirely from it, keeping in mind that all data is based on actual recorded measurements, and that all the vehicles present in the database are EV cars.

In [24]:
df = df.drop(columns=['category', 'mode'])

Additionally, since we are analysing only EV sales and their share in the market overall, it is safe to assume there is more information stored within the database than we need. Hence, it is necessary to clean up the data to keep only what is currently relevant for our study.

In [34]:
# Method A: By keeping only rows where the boolean returns "True"
units_to_remove = ['GWh', 'Milion barrels per day', 'Oil displacement, million lge']
df_units_cleaned = df[~df['unit'].isin(units_to_remove)]
# '~' negates the condition: it keeps only rows NOT containing unwanted units
df_units_cleaned['unit'].unique()

array(['Vehicles', 'percent'], dtype=object)

In [35]:
# Method B: By creating a copy only with the rows we want to keep
ev_parameters = ['EV sales', 'EV stock', 'EV sales share', 'EV stock share']
df_clean = df_units_cleaned[df_units_cleaned['parameter'].isin(ev_parameters)].copy()
df_clean['parameter'].unique()

array(['EV sales', 'EV stock share', 'EV sales share', 'EV stock'],
      dtype=object)

Note: Since both are directly correlated, by using only one of the previous steps the database is already cleant. This is only for demonstration purposes.

In [36]:
df_clean[(df_clean['region']=='Australia') & (df_clean['year']==2012)]

,region,parameter,powertrain,year,unit,value
4,Australia,EV stock,BEV,2012,Vehicles,220.0000
5,Australia,EV stock,PHEV,2012,Vehicles,80.0000
6,Australia,EV sales,PHEV,2012,Vehicles,80.0000
7,Australia,EV sales share,EV,2012,percent,0.0300
8,Australia,EV stock share,EV,2012,percent,0.0024
9,Australia,EV sales,BEV,2012,Vehicles,170.0000


In the previous cell, we can observe that there are two different key units in this database to differenciate — Vehicle counts (physical stock and sales) and Percentages, over the total population of vehicles. The two units cannot be superposed, so it is necessary to separate them in two different sub-dataframes to analyse individually.

In [17]:
df_vehicles = df[df['unit'] == 'Vehicles']
df_percent = df[df['unit'] == 'percent']

Let's see what powertrain counts can be observed in the database for vehicles sold, so we can get an idea of what categories we can find:

In [20]:
df_vehicles['powertrain'].value_counts()

powertrain
BEV     1078
PHEV     922
FCEV     487
Name: count, dtype: int64

If we do the same with the newly created df_percent DF, we can observe that only one category 'EV' appears. That's because an electric vehicle usually comprehends any vehicle that can move fully on battery power, and PHEVs can do so even if the main power is usually the ICE engine. 

In [21]:
df_percent['powertrain'].value_counts()

powertrain
EV    1068
Name: count, dtype: int64

### 1. Vehicle Sales

In [22]:
df_vehicles['parameter'].value_counts()

parameter
EV sales    1342
EV stock    1145
Name: count, dtype: int64

As we can observe, the database contains two distinct parameters for measuring electric vehicles: EV stock and EV sales. EV stock represents the total number of electric vehicles available in a country at a given time, while EV sales represent the number of vehicles actually sold during a specific period. Naturally, sales numbers should always be lower than stock, but analyzing the gap between them over recent years can offer valuable insights into the market’s growth dynamics and saturation levels.

The following distinction will be made, hence, to accomodate these two different data types.

In [30]:
df_vehicles_sales = df_vehicles[df_vehicles['parameter'] == 'EV sales']
df_vehicles_stock = df_vehicles[df_vehicles['parameter'] == 'EV stock']

In [ ]:
df_vehicles_sales

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.0
6,Australia,Historical,EV sales,Cars,PHEV,2012,Vehicles,80.0
9,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.0
10,Australia,Historical,EV sales,Cars,BEV,2013,Vehicles,190.0
13,Australia,Historical,EV sales,Cars,PHEV,2013,Vehicles,100.0
...,...,...,...,...,...,...,...,...
3781,World,Historical,EV sales,Cars,FCEV,2022,Vehicles,15000.0
3783,World,Historical,EV sales,Cars,PHEV,2022,Vehicles,2900000.0
3788,World,Historical,EV sales,Cars,PHEV,2023,Vehicles,4300000.0
3792,World,Historical,EV sales,Cars,FCEV,2023,Vehicles,8900.0


In [ ]:
df_percent_sales = df_percent[df_percent['parameter'] == 'EV sales share']
df_percent_stock = df_percent[df_percent['parameter'] == 'EV stock share']

# 1. Introduction
# 2. Dataset Overview
# 3. Cleaning + Preparation
# 4. Key Questions
# 5. Exploratory Visualizations
# 6. Insights + Observations
# 7. Final Thoughts / Recommendations
